In [1]:
import pandas as pd

# Excelファイルの読み込み
file_path = "weatherdata.xlsx"  # ファイルパスを指定
df = pd.read_excel(file_path)

In [21]:
train_data=[]
for index, row in df.iterrows():
    datad = row.to_dict()
    train_data.append("The weather of " + datad['Unnamed: 0']+ " is "+datad['Unnamed: 1'])


In [31]:
from datasets import load_dataset
from transformers import AutoModelForCausalLM, Trainer, TrainingArguments, AutoTokenizer
import re
from tqdm import tqdm
import torch
from torch.optim import AdamW
import matplotlib.pyplot as plt
from torch import nn
import random

import pandas as pd

# Excelファイルの読み込み
file_path = "weatherdata.xlsx"  # ファイルパスを指定
df = pd.read_excel(file_path)

data=[]
for index, row in df.iterrows():
    datad = row.to_dict()
    data.append("The weather of " + datad['Unnamed: 0']+ " is "+datad['Unnamed: 1']+".")

device='cuda'

size=365
batch=1
epochs=5

teacher_model = AutoModelForCausalLM.from_pretrained("../distillLLAMA2")
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B-Instruct")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id

train_data=[]
for text in tqdm(data, desc="Tokenizing dataset"):
    tokenized = tokenizer(text, padding="max_length", max_length=32, truncation=True, return_tensors="pt")
    input_ids = tokenized['input_ids'].squeeze().tolist()
    attention_mask = tokenized['attention_mask'].squeeze().tolist()
    labels = input_ids[1:] + [tokenizer.pad_token_id]
    for i in range(len(attention_mask)):
        if attention_mask[i]==0:
            labels[i]=-100
    train_data.append({"input_ids": input_ids, "labels": labels, "attention_mask":attention_mask})

input_ids = [item["input_ids"] for item in train_data]
labels = [item["labels"] for item in train_data]
attention_mask = [item["attention_mask"] for item in train_data]

train_data

Tokenizing dataset: 100%|██████████| 365/365 [00:00<00:00, 9176.92it/s]


[{'input_ids': [128000,
   791,
   9282,
   315,
   220,
   16,
   14,
   16,
   374,
   40798,
   13,
   128009,
   128009,
   128009,
   128009,
   128009,
   128009,
   128009,
   128009,
   128009,
   128009,
   128009,
   128009,
   128009,
   128009,
   128009,
   128009,
   128009,
   128009,
   128009,
   128009,
   128009],
  'labels': [791,
   9282,
   315,
   220,
   16,
   14,
   16,
   374,
   40798,
   13,
   128009,
   -100,
   -100,
   -100,
   -100,
   -100,
   -100,
   -100,
   -100,
   -100,
   -100,
   -100,
   -100,
   -100,
   -100,
   -100,
   -100,
   -100,
   -100,
   -100,
   -100,
   -100],
  'attention_mask': [1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0]},
 {'input_ids': [128000,
   791,
   9282,
   315,
   220,
   16,
   14,
   17,
   374,
   40798,
   13,
   128009,
   128009,
   128009,
   128009,
   128009,
   12

In [32]:
from datasets import load_dataset
from transformers import AutoModelForCausalLM, Trainer, TrainingArguments, AutoTokenizer
import re
from tqdm import tqdm
import torch
from torch.optim import AdamW
import matplotlib.pyplot as plt
from torch import nn
import random

import pandas as pd

# Excelファイルの読み込み
file_path = "weatherdata.xlsx"  # ファイルパスを指定
df = pd.read_excel(file_path)

data=[]
for index, row in df.iterrows():
    datad = row.to_dict()
    data.append("The weather of " + datad['Unnamed: 0']+ " is "+datad['Unnamed: 1']+".")

device='cuda'

size=365
batch=1
epochs=5

teacher_model = AutoModelForCausalLM.from_pretrained("../distillLLAMA2")
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B-Instruct")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id

train_data=[]
for text in tqdm(data, desc="Tokenizing dataset"):
    tokenized = tokenizer(text, padding="max_length", max_length=16, truncation=True, return_tensors="pt")
    input_ids = tokenized['input_ids'].squeeze().tolist()
    attention_mask = tokenized['attention_mask'].squeeze().tolist()
    labels = input_ids[1:] + [tokenizer.pad_token_id]
    for i in range(len(attention_mask)):
        if attention_mask[i]==0:
            labels[i]=-100
    train_data.append({"input_ids": input_ids, "labels": labels, "attention_mask":attention_mask})

input_ids = [item["input_ids"] for item in train_data]
labels = [item["labels"] for item in train_data]
attention_mask = [item["attention_mask"] for item in train_data]

print(attention_mask)

Tokenizing dataset: 100%|██████████| 365/365 [00:00<00:00, 10003.67it/s]

[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0],

In [ ]:
from datasets import load_dataset
from transformers import AutoModelForCausalLM, Trainer, TrainingArguments, AutoTokenizer
import re
from tqdm import tqdm
import torch
from torch.optim import AdamW
import matplotlib.pyplot as plt
from torch import nn
import random

import pandas as pd

# Excelファイルの読み込み
file_path = "weatherdata.xlsx"  # ファイルパスを指定
df = pd.read_excel(file_path)

data=[]
for index, row in df.iterrows():
    datad = row.to_dict()
    data.append("The weather of " + datad['Unnamed: 0']+ " is "+datad['Unnamed: 1']+".")

device='cuda'

size=365
batch=1
epochs=1

teacher_model = AutoModelForCausalLM.from_pretrained("weather_model3")
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B-Instruct")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id

train_data=[]
for text in tqdm(data, desc="Tokenizing dataset"):
    tokenized = tokenizer(text, padding="max_length", max_length=16, truncation=True, return_tensors="pt")
    input_ids = tokenized['input_ids'].squeeze().tolist()
    attention_mask = tokenized['attention_mask'].squeeze().tolist()
    labels = input_ids[1:] + [tokenizer.pad_token_id]
    for i in range(len(attention_mask)):
        if attention_mask[i]==0:
            labels[i]=-100
    train_data.append({"input_ids": input_ids, "labels": labels, "attention_mask":attention_mask})

input_ids = [item["input_ids"] for item in train_data]
labels = [item["labels"] for item in train_data]
attention_mask = [item["attention_mask"] for item in train_data]


sbatch = int(size/batch)

batch_train=[]
for i in range(sbatch):
    batch_input=[input_ids[i]]
    batch_train.append(batch_input)
input_ids=batch_train

batch_train=[]
for i in range(sbatch):
    batch_input=[labels[i]]
    batch_train.append(batch_input)
labels=batch_train

batch_train=[]
for i in range(sbatch):
    batch_input=[attention_mask[i]]
    batch_train.append(batch_input)
attention_mask=batch_train


input_ids_tensor = torch.tensor(input_ids, dtype=torch.long)
labels_tensor = torch.tensor(labels, dtype=torch.long)
attention_mask_tensor = torch.tensor(attention_mask, dtype=torch.long)



# クロスエントロピー損失関数の設定
criterion = torch.nn.CrossEntropyLoss(ignore_index=-100)
criterion.to(device)

optimizer = AdamW(teacher_model.parameters(), lr=5e-5)
input_ids_tensor=input_ids_tensor.to(device)
labels_tensor=labels_tensor.to(device)
attention_mask_tensor = attention_mask_tensor.to(device)
teacher_model.to(device)
eval_loss=0
teacher_model.train()

eval_arr = []
label_arr = []
vocab_size = teacher_model.config.vocab_size
for j in range(epochs):
    for i in range(sbatch):
     
        input_ids=input_ids_tensor[i]
        labels=labels_tensor[i]
        attention_mask=attention_mask_tensor[i]
        optimizer.zero_grad()
        outputs = teacher_model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        logits=outputs.logits
        topk_values, topk_indices = torch.topk(logits[0][8], k=1)
        

labels_tensor[0]

Tokenizing dataset: 100%|██████████| 365/365 [00:00<00:00, 9994.13it/s]


tensor([[   791,   9282,    315,    220,     16,     14,     16,    374,  40798,
             13, 128009,   -100,   -100,   -100,   -100,   -100]],
       device='cuda:0')

In [43]:
from datasets import load_dataset
from transformers import AutoModelForCausalLM, Trainer, TrainingArguments, AutoTokenizer
import re
from tqdm import tqdm
import torch
from torch.optim import AdamW
import matplotlib.pyplot as plt
from torch import nn
import random

import pandas as pd

# Excelファイルの読み込み
file_path = "weatherdata.xlsx"  # ファイルパスを指定
df = pd.read_excel(file_path)

data=[]
for index, row in df.iterrows():
    datad = row.to_dict()
    data.append("The weather of " + datad['Unnamed: 1']+ " is "+datad['Unnamed: 1']+".")

device='cuda'

size=365
batch=1
epochs=10

teacher_model = AutoModelForCausalLM.from_pretrained("weather_model")
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B-Instruct")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id

train_data=[]
for text in tqdm(data, desc="Tokenizing dataset"):
    tokenized = tokenizer(text, padding="max_length", max_length=16, truncation=True, return_tensors="pt")
    input_ids = tokenized['input_ids'].squeeze().tolist()
    attention_mask = tokenized['attention_mask'].squeeze().tolist()
    labels = input_ids[1:] + [tokenizer.pad_token_id]
    for i in range(len(attention_mask)):
        if attention_mask[i]==0:
            labels[i]=-100
    train_data.append({"input_ids": input_ids, "labels": labels, "attention_mask":attention_mask})

input_ids = [item["input_ids"] for item in train_data]
labels = [item["labels"] for item in train_data]
attention_mask = [item["attention_mask"] for item in train_data]

print(input_ids[0][6])



Tokenizing dataset: 100%|██████████| 365/365 [00:00<00:00, 9815.23it/s]

40798
